In [9]:
import sympy as sp

x = sp.symbols("x")

def F_m(m):
    return sp.integrate(sp.sin(x)*sp.E**(-sp.I*x*m), (x, -sp.pi, sp.pi))

v, K, alpha0 = sp.symbols(r"v  K \alpha_0")
nabdaWp, nabdaWm = sp.symbols(r"\nabla\mathbf{W}^+, \nabla\mathbf{W}^-")
omegaBarp, omegaBarm = sp.symbols(r"\bar{\omega}^+, \bar{\omega}^-")
varrho2p, varrho2m = sp.symbols(r"\varrho_2^+, \varrho_2^-")
wp, wm = sp.symbols(r"\mathbf{W}^+, \mathbf{W}^-")
rhop, rhom = sp.symbols(r"\rho^+, \rho^-")

eqDotVarrho2p = sp.Eq(
    -v / 2 * nabdaWp + 2 * sp.I * omegaBarp * varrho2p
    + sp.I * 2 * K / 2 / sp.pi * (wp * wp * F_m(1) + sp.E**(-sp.I * alpha0) * wp * wm * F_m(1))
    - sp.I * 2 * K * rhom * varrho2p * sp.sin(alpha0)
, 0)
eqDotVarrho2m = sp.Eq(
    -v / 2 * nabdaWm + 2 * sp.I * omegaBarm * varrho2m
    + sp.I * 2 * K / 2 / sp.pi * (wm * wm * F_m(1) + sp.E**(-sp.I * alpha0) * wm * wp * F_m(1))
    - sp.I * 2 * K * rhop * varrho2m * sp.sin(alpha0)
, 0)
solution = sp.solve((eqDotVarrho2p, eqDotVarrho2m), (varrho2p, varrho2m))

In [2]:
F_m(1)

-I*pi

In [68]:
eqDotVarrho2p
print(sp.latex(eqDotVarrho2p))

- 2 i K \rho^{-} \varrho^{+}_{2} \sin{\left(\alpha_{0} \right)} + \frac{i K \left(- i \pi \left(\mathbf{W}^+\right)^{2} - i \pi \mathbf{W}^+ \mathbf{W}^- e^{- i \alpha_{0}}\right)}{\pi} + 2 i \bar{\omega}^+ \varrho^{+}_{2} - \frac{\nabla\mathbf{W}^+ v}{2} = 0


In [10]:
sp.expand(solution[varrho2p])

-2*I*K*\mathbf{W}^+**2*exp(I*\alpha_0)/(4*K*\rho^-*exp(I*\alpha_0)*sin(\alpha_0) - 4*\bar{\omega}^+*exp(I*\alpha_0)) - 2*I*K*\mathbf{W}^+*\mathbf{W}^-/(4*K*\rho^-*exp(I*\alpha_0)*sin(\alpha_0) - 4*\bar{\omega}^+*exp(I*\alpha_0)) + I*\nabla\mathbf{W}^+*v*exp(I*\alpha_0)/(4*K*\rho^-*exp(I*\alpha_0)*sin(\alpha_0) - 4*\bar{\omega}^+*exp(I*\alpha_0))

In [6]:
solution[varrho2p] = sp.simplify(sp.expand(solution[varrho2p]))
solution[varrho2m] = sp.simplify(sp.expand(solution[varrho2m]))

In [8]:
print(sp.latex(solution[varrho2p]))

\frac{i \left(- 2 K \left(\mathbf{W}^+\right)^{2} e^{i \alpha_{0}} - 2 K \mathbf{W}^+ \mathbf{W}^- + \nabla\mathbf{W}^+ v e^{i \alpha_{0}}\right) e^{- i \alpha_{0}}}{4 \left(K \rho^{-} \sin{\left(\alpha_{0} \right)} - \bar{\omega}^+\right)}


In [5]:
sp.simplify(solution[varrho2m])

I*(-2*K*\mathbf{W}^+*\mathbf{W}^- - 2*K*\mathbf{W}^-**2*exp(I*\alpha_0) + \nabla\mathbf{W}^-*v*exp(I*\alpha_0))*exp(-I*\alpha_0)/(4*(K*\rho^+*sin(\alpha_0) - \bar{\omega}^-))

In [17]:
import sympy as sp

# 定义符号变量
x, y = sp.symbols('x y')

# 定义方程组
eq1 = sp.Eq(x + y, 10)
eq2 = sp.Eq(x - y, 2)

# 解方程组，求解 y 的表达式
solution = sp.solve((eq1, eq2), (x, y))

# 输出结果
y_expression = solution[y]
print(f'y 的表达式: {y_expression}')

y 的表达式: 4
